# retrieval 코드 뜯어보기

In [1]:
import os
import json
import time
import faiss
import pickle
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from contextlib import contextmanager
from typing import List, Tuple, NoReturn, Any, Optional, Union


from sklearn.feature_extraction.text import TfidfVectorizer

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

class SparseRetrieval:
    def __init__(
        self,
        tokenize_fn,
        data_path: Optional[str] = "../data/",
        context_path: Optional[str] = "wikipedia_documents.json",
    ) -> NoReturn:

        """
        Arguments:
            tokenize_fn:
                기본 text를 tokenize해주는 함수입니다.
                아래와 같은 함수들을 사용할 수 있습니다.
                - lambda x: x.split(' ')
                - Huggingface Tokenizer
                - konlpy.tag의 Mecab

            data_path:
                데이터가 보관되어 있는 경로입니다.

            context_path:
                Passage들이 묶여있는 파일명입니다.

            data_path/context_path가 존재해야합니다.

        Summary:
            Passage 파일을 불러오고 TfidfVectorizer를 선언하는 기능을 합니다.
        """

        self.data_path = data_path
        with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
            wiki = json.load(f)

        self.contexts = list(
            dict.fromkeys([v["text"] for v in wiki.values()])
        )  # set 은 매번 순서가 바뀌므로
        print(f"Lengths of unique contexts : {len(self.contexts)}")
        self.ids = list(range(len(self.contexts)))

        # Transform by vectorizer
        self.tfidfv = TfidfVectorizer(
            tokenizer=tokenize_fn,
            ngram_range=(1, 2),
            max_features=50000,
        )

        self.p_embedding = None  # get_sparse_embedding()로 생성합니다
        self.indexer = None  # build_faiss()로 생성합니다.


In [2]:
data_path: Optional[str] = "../data/"
context_path: Optional[str] = "wikipedia_documents.json"

## 기본 tfidf

In [3]:
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()])) # set은 순서가 바뀜
print(f"Lengths of unique contexts : {len(contexts)}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/wikipedia_documents.json'

In [4]:
os.listdir()

['dataset_dict.json',
 'validation',
 'train',
 'test_code.ipynb',
 'sparse_embedding.bin',
 'tfidv.bin']

In [ ]:
print(contexts[0])

이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.
# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.

두 목록은 모두 가나다 순이다.

일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.


In [ ]:
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "seongju/klue-mrc-koelectra-base",
    use_fast=True,
)

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='seongju/klue-mrc-koelectra-base', vocab_size=35000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
tokenizer.tokenize

<bound method PreTrainedTokenizerFast.tokenize of PreTrainedTokenizerFast(name_or_path='seongju/klue-mrc-koelectra-base', vocab_size=35000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>

In [ ]:
tfidfv = TfidfVectorizer(
    tokenizer=tokenizer.tokenize,
    ngram_range=(1, 2),
    max_features=50000,
)
print(tfidfv)

TfidfVectorizer(max_features=50000, ngram_range=(1, 2),
                tokenizer=<bound method PreTrainedTokenizerFast.tokenize of PreTrainedTokenizerFast(name_or_path='seongju/klue-mrc-koelectra-base', vocab_size=35000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>)


## get_sparse_embedding 함수

In [ ]:
"""
Summary:
    Passage Embedding을 만들고
    TFIDF와 Embedding을 pickle로 저장합니다.
    만약 미리 저장된 파일이 있으면 저장된 pickle을 불러옵니다.
"""
p_embedding = None  # get_sparse_embedding()로 생성합니다
indexer = None  # build_faiss()로 생성합니다.
# Pickle을 저장합니다.

pickle_name = f"sparse_embedding.bin"
tfidfv_name = f"tfidv.bin"
emd_path = os.path.join(data_path, pickle_name)
tfidfv_path = os.path.join(data_path, tfidfv_name)

if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv = pickle.load(file)
    print("Embedding pickle load.")
else:
    print("Build passage embedding")
    p_embedding = tfidfv.fit_transform(contexts)
    print(p_embedding.shape)
    with open(emd_path, "wb") as file:
        pickle.dump(p_embedding, file)
    with open(tfidfv_path, "wb") as file:
        pickle.dump(tfidfv, file)
    print("Embedding pickle saved.")

Embedding pickle load.


In [ ]:
# Context 갯수가 Row의 총 갯수
# 그리고 기존에 max_features = 50,000 으로 두었기 때문에 이렇게 진행된다.
p_embedding 

<56737x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 18906074 stored elements in Compressed Sparse Row format>